# Create DuckDB database

In [ ]:
import pandas as pd
df_data_dict = pd.read_csv('../data/extracted_table.csv', skiprows=1)
all_variable_names = df_data_dict['Variable Name'].dropna().tolist()
all_variable_names = [name for name in all_variable_names if name != '']
print(f"Length of all_variable_names: {len(all_variable_names)}")
all_variable_names

Length of all_variable_names: 22


['Prscrbr_NPI',
 'Prscrbr_Last_Org_Name',
 'Prscrbr_First_Name',
 'Prscrbr_City',
 'Prscrbr_State_Abrvtn',
 'Prscrbr_State_FIPS',
 'Prscrbr_Type',
 'Prscrbr_Type_Src',
 'Brnd_Name',
 'Gnrc_Name',
 'Tot_Clms',
 'Tot_30day_Fills',
 'Tot_Day_Suply',
 'Tot_Drug_Cst',
 'Tot_Benes',
 'GE65_Sprsn_Flag',
 'GE65_Tot_Clms',
 'GE65_Tot_30day_Fills',
 'GE65_Tot_Drug_Cst',
 'GE65_Tot_Day_Suply',
 'GE65_Bene_Sprsn_Flag',
 'GE65_Tot_Benes']

In [6]:
df_data = pd.read_csv('../data/Medicare Part D Prescribers - by Provider and Drug/2022/MUP_DPR_RY24_P04_V10_DY22_NPIBN.csv',nrows=100, usecols=all_variable_names)
df_data

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,...,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes
0,1003000126,Enkeshafi,Ardalan,Bethesda,MD,24,Internal Medicine,Claim-Specialty,Amlodipine Besylate,Amlodipine Besylate,...,1230,246.17,14.0,NaN,19.0,41.0,246.17,1230.0,NaN,14.0
1,1003000126,Enkeshafi,Ardalan,Bethesda,MD,24,Internal Medicine,Claim-Specialty,Atorvastatin Calcium,Atorvastatin Calcium,...,870,286.69,NaN,NaN,11.0,29.0,286.69,870.0,*,NaN
2,1003000126,Enkeshafi,Ardalan,Bethesda,MD,24,Internal Medicine,Claim-Specialty,Eliquis,Apixaban,...,508,9503.53,NaN,NaN,15.0,17.0,9503.53,508.0,*,NaN
3,1003000126,Enkeshafi,Ardalan,Bethesda,MD,24,Internal Medicine,Claim-Specialty,Escitalopram Oxalate,Escitalopram Oxalate,...,783,277.71,NaN,NaN,16.0,26.2,277.71,783.0,*,NaN
4,1003000126,Enkeshafi,Ardalan,Bethesda,MD,24,Internal Medicine,Claim-Specialty,Hydrochlorothiazide,Hydrochlorothiazide,...,542,68.18,NaN,#,NaN,NaN,NaN,NaN,*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1003000530,Semonche,Amanda,Quakertown,PA,42,Internal Medicine,Claim-Specialty,Lantus Solostar,"Insulin Glargine,hum.Rec.Anlog",...,1128,11914.18,NaN,NaN,35.0,47.5,11914.18,1128.0,*,NaN
96,1003000530,Semonche,Amanda,Quakertown,PA,42,Internal Medicine,Claim-Specialty,Levothyroxine Sodium,Levothyroxine Sodium,...,15768,3783.79,49.0,NaN,225.0,478.0,3161.36,14278.0,#,NaN
97,1003000530,Semonche,Amanda,Quakertown,PA,42,Internal Medicine,Claim-Specialty,Lisinopril,Lisinopril,...,21295,2690.57,70.0,NaN,268.0,676.1,2633.48,20200.0,#,NaN
98,1003000530,Semonche,Amanda,Quakertown,PA,42,Internal Medicine,Claim-Specialty,Lisinopril-Hydrochlorothiazide,Lisinopril/Hydrochlorothiazide,...,4140,878.21,12.0,NaN,48.0,138.0,878.21,4140.0,NaN,12.0


In [2]:
# Create a DuckDB connection and save the dataframe to a database
import duckdb


con = duckdb.connect('medicare_data.duckdb')
con.execute(f"CREATE OR REPLACE TABLE medicare_data AS SELECT {', '.join(all_variable_names)} FROM read_csv_auto('../data/Medicare Part D Prescribers - by Provider and Drug/2022/MUP_DPR_RY24_P04_V10_DY22_NPIBN.csv')")
con.close()


# Find Prescribers from University of Washington

This query will connect to the DuckDB database and retrieve prescribers whose organization contains 'University of Washington'.

In [7]:
# Connect to the DuckDB database
import duckdb

# Open a connection to the database
con = duckdb.connect('medicare_data.duckdb')

# Query for prescribers from University of Washington
query = """
SELECT * 
FROM medicare_data 
WHERE LOWER(Prscrbr_Last_Org_Name) LIKE '%university of washington%'
LIMIT 20
"""

# Execute the query and display the results
result = con.execute(query).fetchdf()
count_query = "SELECT COUNT(*) FROM medicare_data WHERE LOWER(Prscrbr_Last_Org_Name) LIKE '%university of washington%'"
count_result = con.execute(count_query).fetchone()[0]
print(f"Found {count_result} prescribers from University of Washington")
result

# Close the connection
con.close()

Found 0 prescribers from University of Washington


## NPPES data

In [3]:
import pandas as pd
import duckdb
import os

# Folder containing the files
folder_path = '../data/NPPES_Data_Dissemination_April_2025_V2'

# Iterate through files in the folder
for file in os.listdir(folder_path):
    if file.endswith('_fileheader.csv'):
        # Read the fileheader to get column names
        fileheader_path = os.path.join(folder_path, file)
        column_names = pd.read_csv(fileheader_path).columns.tolist()
        
        # Get the corresponding data file
        data_file = file.replace('_fileheader.csv', '.csv')
        data_file_path = os.path.join(folder_path, data_file)
        
        if os.path.exists(data_file_path):
            # Create a DuckDB table using the column names
            table_name = os.path.splitext(data_file)[0]
            con = duckdb.connect('medicare_data.duckdb')
            # Quote column names to handle spaces or special characters
            quoted_column_names = [f'"{col}"' for col in column_names]
            
        try:
            con.execute(f"""
                CREATE OR REPLACE TABLE "{table_name}" AS 
                SELECT * FROM read_csv_auto(
                    '{data_file_path}',
                    all_varchar=True
                )
            """)
        except Exception as e:
            print(f"Error processing file {data_file_path}: {e}")
            con.execute(f"""
                CREATE OR REPLACE TABLE "{table_name}" AS 
                SELECT * FROM read_csv_auto(
                    '{data_file_path}',
                    all_varchar=True,
                    ignore_errors=True
                )
            """)
            con.close()